# Exception Patterns and Best Practices

**Chapter 6 - Learning Python, 5th Edition**

Production-ready exception patterns: retry logic, graceful degradation,
and warnings for non-fatal issues.

## Retry Pattern with Exponential Backoff

In [ ]:
import time
import random
from typing import TypeVar, Callable

T = TypeVar("T")


def retry(
    func: Callable[[], T],
    max_attempts: int = 3,
    base_delay: float = 0.1,
    exceptions: tuple[type[Exception], ...] = (Exception,),
) -> T:
    """Retry a function with exponential backoff."""
    last_exception: Exception | None = None

    for attempt in range(1, max_attempts + 1):
        try:
            return func()
        except exceptions as e:
            last_exception = e
            if attempt < max_attempts:
                delay = base_delay * (2 ** (attempt - 1))
                print(f"  Attempt {attempt} failed: {e}. Retrying in {delay:.2f}s...")
                time.sleep(delay)

    raise last_exception  # type: ignore[misc]


# Simulate a flaky operation
call_count = 0


def flaky_operation() -> str:
    global call_count
    call_count += 1
    if call_count < 3:
        raise ConnectionError(f"Connection refused (attempt {call_count})")
    return "Success!"


result = retry(flaky_operation, max_attempts=5, base_delay=0.01)
print(f"Result: {result}")

## Graceful Degradation with Fallbacks

In [ ]:
def with_fallback(*funcs: Callable[[], T]) -> T:
    """Try each function in order, returning the first success."""
    errors: list[Exception] = []

    for func in funcs:
        try:
            return func()
        except Exception as e:
            errors.append(e)

    raise ExceptionGroup("All fallbacks failed", errors)


def primary_source() -> dict:
    raise ConnectionError("Primary DB unavailable")


def cache_source() -> dict:
    raise FileNotFoundError("Cache miss")


def default_source() -> dict:
    return {"status": "default", "data": []}


result = with_fallback(primary_source, cache_source, default_source)
print(f"Got data from fallback: {result}")

## Warnings for Non-Fatal Issues

Use `warnings` for issues that shouldn't stop execution but should be noticed:

In [ ]:
import warnings


class DeprecatedFeatureWarning(UserWarning):
    """Warning for deprecated features."""


def old_api(data: str) -> str:
    """A deprecated function that still works."""
    warnings.warn(
        "old_api() is deprecated, use new_api() instead",
        DeprecatedFeatureWarning,
        stacklevel=2,
    )
    return data.upper()


# Capture warnings for demonstration
with warnings.catch_warnings(record=True) as caught:
    warnings.simplefilter("always")
    result = old_api("hello")
    print(f"Result: {result}")
    print(f"Warning: {caught[0].category.__name__}: {caught[0].message}")

## Context Manager for Error Handling

Combining context managers with exception handling for clean resource management:

In [ ]:
from contextlib import contextmanager


@contextmanager
def error_boundary(operation: str, default=None):
    """Context manager that catches and logs errors, providing a default."""
    result = {"value": default, "error": None}
    try:
        yield result
    except Exception as e:
        result["error"] = e
        print(f"[{operation}] Error suppressed: {type(e).__name__}: {e}")


# Error is caught and suppressed
with error_boundary("division", default=0) as ctx:
    ctx["value"] = 10 / 0  # ZeroDivisionError caught

print(f"Result: {ctx['value']}, Error: {ctx['error']}")

print()

# Normal operation
with error_boundary("division", default=0) as ctx:
    ctx["value"] = 10 / 3

print(f"Result: {ctx['value']:.2f}, Error: {ctx['error']}")

## Best Practices Summary

1. **Catch specific exceptions**, never bare `except:`
2. **Use EAFP** over LBYL for Pythonic code
3. **Chain exceptions** with `raise ... from` to preserve context
4. **Create exception hierarchies** for your domain
5. **Use `finally`** for guaranteed cleanup
6. **Use `else`** for code that should only run on success
7. **Log and re-raise** rather than silently swallowing errors
8. **Use warnings** for non-fatal issues